In [25]:
%cd /content/drive/MyDrive/exp_2023_onward

/content/drive/MyDrive/exp_2023_onward


# **Project Overview**:
This project is dedicated to enhancing the FlanT5 model for dialogue summarization. We aim to achieve this through a two-fold approach: firstly, by fine-tuning its parameters effectively, and secondly, by implementing the Reinforcement Learning for Hate Speech Filtering (RLHF) framework to ensure the generated summaries are free from toxic content. Our primary objective is the refinement of the FLAN-T5 model.

Key Steps:
1. **Dataset Acquisition and Preparation**
   - Our project kicks off with the acquisition of the dataset, followed by meticulous preparation for the subsequent training phase.

2. **Model Initialization and Efficient Parameter Fine-Tuning**
   - The project's next phase entails initializing the model and optimizing it for streamlined parameter fine-tuning.

3. **Model Evaluation using Rouge Score**
   - Following efficient parameter fine-tuning, its performance is evaluated using the Rouge score, which measures the quality of the generated summaries.

4. **Toxicity Assessment of Generated Summaries**
   - We employ the 'roberta-hate-speech' model to assess the toxicity of the generated summaries, ensuring they meet non-toxicity standards.

5. **Perform Fine-Tuning using RLHF Framework to Detoxify the Summaries**
   - The RLHF fine-tuning process involves the following components:
     - PPO Model: This model, which undergoes optimized fine-tuning, aims to improve non-toxicity in the summaries.
     - Reference Model: A frozen model used to calculate the KL-divergence from the initial model state. It provides an additional reward signal during PPO training to prevent significant deviations from the original Language Model (LLM).
     - Score Generator Model: This model is responsible for generating non-toxicity scores for the summaries.

6. **Evaluation of Non-Toxicity Performance Enhancement**
   - Finally, we assess the improvements in non-toxicity achieved by the RLHF fine-tuned model to ensure the generated summaries meet high standards of safety and quality.

In [1]:
!pip install transformers peft datasets evaluate
!pip install git+https://github.com/lvwerra/trl.git@25fa1bd


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.1 MB/s eta 0:00:00
  Cloning https://github.com/lvwerra/trl.git (to revision 25fa1bd) to /tmp/pip-req-build-mase63ba
  Run

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig, Trainer, TrainingArguments
from datasets import load_dataset
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

import torch
import evaluate

import numpy as np
import pandas as pd
from tqdm import tqdm


### **Section 1: Load Dataset and pre-process for Finetuning**.




In [3]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset_original = load_dataset(huggingface_dataset_name)
dataset_original

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [4]:

def preprocess_and_split_dataset(tokenizer_name,
                                 dataset_to_load,
                                 min_length,
                                 ):

    # Load the dataset, using only the "train" split for this task.
    dataset = load_dataset(dataset_to_load, split="train")

    # Filter dialogues with lengths within the specified range.
    dataset = dataset.filter(lambda x: len(x["dialogue"]) > min_length,batched=False)

    # Initialize the tokenizer, automatically switching between GPU and CPU.

    def tokenize_function(example):
      start_prompt = 'Summarize the following conversation.\n\n'
      end_prompt = '\n\nSummary: '
      prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
      example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
      example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
      #example["query"] = tokenizer.decode(example["input_ids"][0])
      return example


    # Tokenize each dialogue in the dataset.
    dataset = dataset.map(tokenize_function, batched=True)
    dataset.set_format(type="torch")

    # Split the dataset into train and test sections.
    dataset_splits = dataset.train_test_split(test_size=0.2, shuffle=False, seed=42)

    return dataset_splits

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

dataset_name = huggingface_dataset_name  # Replace with the actual dataset name
min_length = 200
max_length = 1000

dataset = preprocess_and_split_dataset(tokenizer,
                                       dataset_to_load=dataset_name,
                                       min_length=min_length)

print(dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 9964
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 2492
    })
})


### **Section 2: Model Initialization and Efficient Parameter Fine-Tuning.**


In [23]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
peft_model = get_peft_model(original_model,
                            lora_config)


In [24]:
import time
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-4, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
)


peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset['test']
)

In [25]:
peft_trainer.train()

peft_model_path="./peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
500,6.205400
1000,0.248700
1500,0.187800
2000,0.170600


('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

### **Section 3: Model Evaluation using Rouge Score**

In [34]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base",
                                                        torch_dtype=torch.bfloat16,
                                                        device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       'peft-dialogue-summary-checkpoint-local',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False,
                                       )

In [33]:
index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids.to(original_model.device), generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids.to(peft_model.device), generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}\n')
print(f'ORIGINAL MODEL:\n{original_model_text_output}\n')
print(f'PEFT MODEL: {peft_model_text_output}\n')

BASELINE HUMAN SUMMARY:
#Person2# shows #Person1# the way to the post office.

ORIGINAL MODEL:
#Person1# tells #Person1# where the post office is on the Fifth Avenue.

PEFT MODEL: #Person1# asks #Person2# where the post office is on Fifth Avenue. #Person1# tells #Person2# to walk two blocks ahead, then turn left.



In [35]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
              Summarize the following conversation.

              {dialogue}

              Summary: """



    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids.to(original_model.device), generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids.to(peft_model.device), generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries','original_model_summaries', 'peft_model_summaries'])

df.head()

Token indices sequence length is longer than the specified maximum sequence length for this model (545 > 512). Running this sequence through the model will result in indexing errors


,human_baseline_summaries,original_model_summaries,peft_model_summaries
0,#Person1# and #Person2# are talking about #Per...,#Person1# tells the story of Ricky Martin's si...,"#Person1# was happy that his favorite singer, ..."
1,#Person2# is doing business through the Intern...,#Person1# wants to do business through Interne...,#Person2# is doing business through Internet n...
2,#Person1# has to miss school next week and ass...,#Person1# has to miss school next week. #Perso...,#Person1# has to miss school next week. #Perso...
3,#Person1# and #Person2# will meet Conrad at a ...,#Person1# and #Person2# are going to pick Conr...,#Person1# and #Person2# are going to meet Conr...
4,Maria worries that her mother won't allow her ...,Maria's mother won't allow her to accept a par...,Maria's mother won't allow her to accept a par...


In [29]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install rouge_score

In [36]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.34123586883765394, 'rouge2': 0.1440017083888651, 'rougeL': 0.2831497924525362, 'rougeLsum': 0.2807635096977867}
PEFT MODEL:
{'rouge1': 0.4007367787331477, 'rouge2': 0.18594748933648575, 'rougeL': 0.32906372654516264, 'rougeLsum': 0.333225720958218}


### **Section 4: Toxicity Assessment of Generated Summaries**


In [12]:
class ToxicModel:
    def __init__(self,toxicity_model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(toxicity_model_name, device_map="auto")
        self.model  = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name, device_map="auto")

    def __call__(self, text,return_probs=False):
        input_ids = self.tokenizer(text, padding=True, truncation=True,return_tensors="pt").input_ids
        with torch.no_grad():
            logits = self.model(input_ids.to(self.model.device)).logits

        if return_probs:
            probabilities = logits.softmax(dim=-1).tolist()
            return probabilities
        else:
            not_hate_index = 0
            score= logits[:, not_hate_index].tolist()
            return score

toxicity_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_evaluator = ToxicModel(toxicity_model_name)



In [13]:
print("Non toxic example: ")
non_toxic_text = "#Person 1# tells Tommy that he didn't like the movie."
print("Non toxic Probablity",toxicity_evaluator(non_toxic_text,return_probs=True))
print("Non toxic Reward",toxicity_evaluator(non_toxic_text,return_probs=False))

print("Toxic example: ")
toxic_text= "#Person 1# tells Tommy that the movie was terrible, dumb and stupid."
print("Toxic Probablity",toxicity_evaluator(toxic_text,return_probs=True))
print("Toxic Reward",toxicity_evaluator(toxic_text,return_probs=False))


Non toxic example: 
Non toxic Probablity [[0.9963293671607971, 0.0036706167738884687]]
Non toxic Reward [3.114100694656372]
Toxic example: 
Toxic Probablity [[0.2564719617366791, 0.7435280084609985]]
Toxic Reward [-0.6921164393424988]


In [14]:
def evaluate_toxicity(model,
                      toxicity_evaluator,
                      tokenizer,
                      dataset,
                      num_samples):

    """
    Preprocess the dataset and split it into train and test parts.

    Parameters:
    - model (trl model): Model to be evaluated.
    - toxicity_evaluator (evaluate_modules toxicity metrics): Toxicity evaluator.
    - tokenizer (transformers tokenizer): Tokenizer to be used.
    - dataset (dataset): Input dataset for the evaluation.
    - num_samples (int): Maximum number of samples for the evaluation.

    Returns:
    tuple: A tuple containing two numpy.float64 values:
    - mean (numpy.float64): Mean of the samples toxicity.
    - std (numpy.float64): Standard deviation of the samples toxicity.
    """

    max_new_tokens=100

    toxicities = []
    input_texts = []
    for i, sample in tqdm(enumerate(dataset)):
        start_prompt = 'Summarize the following conversation.\n\n'
        end_prompt = '\n\nSummary: '
        input_text = start_prompt + sample["dialogue"] + end_prompt

        if i > num_samples:
            break

        input_ids = tokenizer(input_text, return_tensors="pt", padding=True).input_ids

        generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
                                             top_k=0.0,
                                             top_p=1.0,
                                             do_sample=True)
        try:
          input_ids=input_ids.to(model.device)
        except:
          input_ids=input_ids.to(model.current_device)
        response_token_ids = model.generate(input_ids=input_ids,
                                            generation_config=generation_config)

        generated_text = tokenizer.decode(response_token_ids[0], skip_special_tokens=True)

        toxicity_score = toxicity_evaluator(generated_text)

        toxicities.extend(toxicity_score)

    # Compute mean & std using np.
    mean = np.mean(toxicities)
    std = np.std(toxicities)

    return mean, std

tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")
mean_before_detoxification, std_before_detoxification = evaluate_toxicity(model=peft_model,
                                                                          toxicity_evaluator=toxicity_evaluator,
                                                                          tokenizer=tokenizer,
                                                                          dataset=dataset["test"],
                                                                          num_samples=10)

print(f'toxicity [mean, std] before detox: [{mean_before_detoxification}, {std_before_detoxification}]')

3it [00:03,  1.06s/it]Token indices sequence length is longer than the specified maximum sequence length for this model (545 > 512). Running this sequence through the model will result in indexing errors
11it [00:15,  1.43s/it]

toxicity [mean, std] before detox: [2.3352706134319305, 1.390401880705651]


### **Section 5: Perform Fine-Tuning using RLHF Framework to Detoxify the Summaries.**


The fine-tuning loop consists of the following main steps:
1. Get the query responses from the policy LLM (PEFT model).
2. Get sentiments for query/responses from hate speech RoBERTa model.
3. Optimize policy with PPO using the (query, response, reward) triplet.

The operation is running if you see the following metrics appearing:
* `objective/kl`: minimize kl divergence,
* `ppo/returns/mean`: maximize mean returns,
* `ppo/policy/advantages_mean`: maximize advantages.

In [15]:
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model,
                                                               torch_dtype=torch.bfloat16,
                                                               is_trainable=True,
                                                               )

ref_model = create_reference_model(ppo_model)


In [16]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

learning_rate=1.41e-5
max_ppo_epochs=1
mini_batch_size=4
batch_size=16

config = PPOConfig(
    model_name=model_name,
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)

ppo_trainer = PPOTrainer(config=config,
                         model=ppo_model,
                         ref_model=ref_model,
                         tokenizer=tokenizer,
                         dataset=dataset["train"],
                         data_collator=collator)

In [19]:
output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

max_ppo_steps = 10

for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    # Break when you reach max_steps.
    if step >= max_ppo_steps:
        break

    prompt_tensors = batch["input_ids"]

    # Get response from FLAN-T5/PEFT LLM.
    summary_tensors = []

    for prompt_tensor in prompt_tensors:
        max_new_tokens = output_length_sampler()

        generation_kwargs["max_new_tokens"] = max_new_tokens
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)

        summary_tensors.append(summary.squeeze()[-max_new_tokens:])

    # This needs to be called "response".
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in summary_tensors]

    # Compute reward outputs.
    rewards = toxicity_evaluator(batch["response"])
    rewards=[torch.tensor(r) for r in rewards]
    # You use the `nothate` item because this is the score for the positive `nothate` class.

    # Run PPO step.
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))

1it [00:25, 25.41s/it]

objective/kl: 27.78833770751953
ppo/returns/mean: -0.7307109832763672
ppo/policy/advantages_mean: -1.1751219997080398e-09
---------------------------------------------------------------------------------------------------


2it [00:51, 25.68s/it]

objective/kl: 32.53301239013672
ppo/returns/mean: -0.9490570425987244
ppo/policy/advantages_mean: 5.699219762789198e-09
---------------------------------------------------------------------------------------------------


3it [01:19, 26.91s/it]

...

objective/kl: 41.561058044433594
ppo/returns/mean: -1.2687175273895264
ppo/policy/advantages_mean: 3.6715286366728606e-09
---------------------------------------------------------------------------------------------------


9it [04:19, 30.31s/it]

objective/kl: 33.37474060058594
ppo/returns/mean: -0.9183297157287598
ppo/policy/advantages_mean: 4.912426909697842e-09
---------------------------------------------------------------------------------------------------


10it [04:42, 28.23s/it]

objective/kl: 26.23990249633789
ppo/returns/mean: -0.7045215368270874
ppo/policy/advantages_mean: 6.8878613923573084e-09
---------------------------------------------------------------------------------------------------


### **Section 6:Evaluation of Non-Toxicity Performance Enhancement**


In [20]:
mean_after_detoxification, std_after_detoxification = evaluate_toxicity(model=ppo_model,
                                                                        toxicity_evaluator=toxicity_evaluator,
                                                                        tokenizer=tokenizer,
                                                                        dataset=dataset["test"],
                                                                        num_samples=10)
print(f'toxicity [mean, std] after detox: [{mean_after_detoxification}, {std_after_detoxification}]')
mean_improvement = (mean_before_detoxification - mean_after_detoxification) / mean_before_detoxification
std_improvement = (std_before_detoxification - std_after_detoxification) / std_before_detoxification

print(f'Percentage improvement of toxicity score after detoxification:')
print(f'mean: {mean_improvement*100:.2f}%')
print(f'std: {std_improvement*100:.2f}%')

11it [00:19,  1.76s/it]

toxicity [mean, std] after detox: [2.2445485591888428, 1.1239272173089407]
Percentage improvement of toxicity score after detoxification:
mean: 3.88%
std: 19.17%
